In [1]:
import os
import glob
import re
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

c:\Users\user\miniconda3\envs\env311_cu121\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('C:/wanted/Git_project/DACON-construction-accident-prevention/data/ref_train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('C:/wanted/Git_project/DACON-construction-accident-prevention/data/test.csv', encoding = 'utf-8-sig')

In [3]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [4]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [ ]:
def extract_metadata_from_filename(filename):
    """파일명에서 메타데이터 추출"""
    # 파일명에서 확장자 제거
    filename = os.path.splitext(filename)[0]
    
    # 공사 유형 분류
    construction_types = {
        '건축': ['건축물', '건설공사', '건설현장', '건설기계', '건설업체'],
        '토목': ['교량', '터널', '도로', '철도', '항만', '하천'],
        '조경': ['조경', '수목', '식재'],
        '설비': ['설비', '플랜트', '시스템', '기계'],
        '기타': []
    }
    
    # 작업 유형 분류
    work_types = {
        '기초공사': ['기초', '말뚝', '지하', '굴착'],
        '구조공사': ['철골', '콘크리트', '거푸집', '동바리'],
        '설비공사': ['설비', '배관', '전기', '용접'],
        '마감공사': ['미장', '타일', '도배', '페인트'],
        '기타': []
    }
    
    # 위험 작업 분류
    hazard_types = {
        '고소작업': ['고소', '추락', '비계', '발판'],
        '중량물작업': ['크레인', '양중', '중량물'],
        '굴착작업': ['굴착', '터널', '지하'],
        '전기작업': ['전기', '감전'],
        '기타': []
    }
    
    # 메타데이터 초기화
    metadata = {
        'construction_type': '기타',
        'work_type': '기타',
        'hazard_type': '기타',
        'specific_method': '',  # 특정 공법 (예: NATM, TBM 등)
        'equipment': '',        # 사용 장비
        'location': ''          # 작업 위치
    }
    
    # 공사 유형 분류
    for const_type, keywords in construction_types.items():
        if any(keyword in filename for keyword in keywords):
            metadata['construction_type'] = const_type
            break
    
    # 작업 유형 분류
    for work_type, keywords in work_types.items():
        if any(keyword in filename for keyword in keywords):
            metadata['work_type'] = work_type
            break
    
    # 위험 작업 분류
    for hazard_type, keywords in hazard_types.items():
        if any(keyword in filename for keyword in keywords):
            metadata['hazard_type'] = hazard_type
            break
    
    # 특정 공법 추출
    method_patterns = [
        r'\((.*?)\)',  # 괄호 안의 내용
        r'공법',       # '공법' 앞의 내용
        r'방법'        # '방법' 앞의 내용
    ]
    
    for pattern in method_patterns:
        match = re.search(pattern, filename)
        if match:
            metadata['specific_method'] = match.group(1)
            break
    
    # 장비 추출
    equipment_keywords = ['크레인', '비계', '동바리', '거푸집', '발판', '비계']
    for keyword in equipment_keywords:
        if keyword in filename:
            metadata['equipment'] = keyword
            break
    
    return metadata

def extract_sections(text):
    """텍스트를 섹션별로 분리"""
    sections = []
    current_section = ""
    
    for line in text.split('\n'):
        # 섹션 시작 패턴 (숫자로 시작하는 줄)
        if re.match(r'^\d+\.\s+', line):
            if current_section:
                sections.append(current_section.strip())
            current_section = line
        else:
            current_section += "\n" + line
    
    if current_section:
        sections.append(current_section.strip())
    
    return sections

def load_and_process_documents(txt_path):
    """텍스트 파일들을 로드하고 처리"""
    txt_files = glob.glob(os.path.join(txt_path, '*.txt'))
    documents = []
    
    for txt_file in tqdm(txt_files, desc="문서 로드 중"):
        try:
            with open(txt_file, 'r', encoding='utf-8') as f:
                content = f.read()
                filename = os.path.basename(txt_file)
                
                # 메타데이터 추출
                metadata = extract_metadata_from_filename(filename)
                metadata['filename'] = filename
                
                # 문서 구조화
                document = {
                    "metadata": metadata,
                    "content": content,
                    "sections": extract_sections(content)
                }
                documents.append(document)
        except Exception as e:
            print(f"Error processing {txt_file}: {str(e)}")
    
    return documents

def create_vector_database(documents, model_name='jhgan/ko-sbert-sts'):
    """벡터 데이터베이스 생성"""
    print("임베딩 모델 로드 중...")
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    # 텍스트 분할기 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    # 문서 분할 및 벡터 저장소 생성
    print("문서 분할 및 벡터 저장소 생성 중...")
    texts = []
    metadatas = []
    
    for doc in documents:
        # 텍스트 분할
        splits = text_splitter.split_text(doc["content"])
        texts.extend(splits)
        
        # 각 분할된 텍스트에 대한 메타데이터 복제
        for _ in splits:
            metadatas.append(doc["metadata"])
    
    # FAISS 벡터 저장소 생성
    vectorstore = FAISS.from_texts(
        texts=texts,
        embedding=embeddings,
        metadatas=metadatas
    )
    
    return vectorstore

def save_vector_database(vectorstore, save_path):
    """벡터 데이터베이스 저장"""
    print("벡터 데이터베이스 저장 중...")
    vectorstore.save_local(save_path)

def load_vector_database(load_path, model_name='jhgan/ko-sbert-sts'):
    """벡터 데이터베이스 로드"""
    print("벡터 데이터베이스 로드 중...")
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    vectorstore = FAISS.load_local(load_path, embeddings)
    return vectorstore

def search_similar_sections(query, vectorstore, k=5):
    """유사한 섹션 검색"""
    # 유사도 검색
    results = vectorstore.similarity_search_with_score(query, k=k)
    
    # 결과 반환
    formatted_results = []
    for doc, score in results:
        formatted_results.append({
            'section': doc.page_content,
            'similarity': 1 - score,  # 거리를 유사도로 변환
            'metadata': doc.metadata
        })
    
    return formatted_results

def main():
    # 경로 설정
    txt_path = r"C:\wanted\Git_project\DACON-construction-accident-prevention\data\text_output"
    vector_db_path = r"C:\wanted\Git_project\DACON-construction-accident-prevention\code\JaeSik\db"
    
    # 벡터 DB 디렉토리 생성
    os.makedirs(vector_db_path, exist_ok=True)
    
    # 문서 로드 및 처리
    documents = load_and_process_documents(txt_path)
    print(f"총 {len(documents)}개의 문서가 로드되었습니다.")
    
    # 벡터 데이터베이스 생성
    vectorstore = create_vector_database(documents)
    
    # 벡터 데이터베이스 저장
    save_vector_database(vectorstore, vector_db_path)

    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    llm = ChatOllama(model='EEVE-korean-10.8B')

    template = """
    ### 지침: 당신은 건설 안전 전문가입니다.
    질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
    - 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
    - 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

    ###예시
    고소작업 시 추락 위험이 있는 부위에 안전장비 설치.
    재발 방지 대책 마련과 안전교육 실시.
    자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 후 장애물 사전 정리 작업 실시.

    {context}

    ### 질문:
    {question}

    [/INST]

    """

    prompt = ChatPromptTemplate.from_template(template)

    def format_retrieved_docs(docs):
        """retriever가 반환한 문서를 문자열로 변환"""
        return "\n\n".join([doc.page_content for doc in docs])

    rag_chain = (
        {
            'context': lambda inputs: format_retrieved_docs(retriever.get_relevant_documents(inputs['question'])),
            'question': itemgetter("question")
        }
        | prompt 
        | llm
        | StrOutputParser()
    )
    
    # 테스트 실행 및 결과 저장
    test_results = []

    print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

    for idx, row in combined_test_data.iterrows():
        # 50개당 한 번 진행 상황 출력
        # if (idx + 1) % 50 == 0 or idx == 0:
        #     print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")

        # RAG 체인 호출 및 결과 생성
        prevention_result = rag_chain.invoke({"question" : row['question']})
        print(prevention_result)
        if idx == 3:
            break

        # 결과 저장
        # result_text = prevention_result['result']
        test_results.append(prevention_result)

    print("\n테스트 실행 완료! 총 결과 수:", len(test_results))
    
    return test_results

In [14]:
combined_training_data.iloc[4,1]

'자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 후 장애물 사전 정리 작업 실시.'

In [ ]:
from sentence_transformers import SentenceTransformer
test_results = main()
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv('C:/wanted/Git_project/DACON-construction-accident-prevention/data/sample_submission.csv', encoding = 'utf-8-sig')
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')

In [15]:
txt_path = "C:/wanted/Git_project/DACON-construction-accident-prevention/data/text_output"
txt_files = glob.glob(os.path.join(txt_path, '*.txt'))
extract_metadata_from_filename(txt_files[11])

{'construction_type': '건축',
 'work_type': '기타',
 'hazard_type': '기타',
 'specific_method': '',
 'equipment': '',
 'location': ''}

In [10]:
def save_vector_database(vectorstore, save_path):
    """벡터 데이터베이스 저장"""
    print("벡터 데이터베이스 저장 중...")
    vectorstore.save_local(save_path)

In [16]:
docs = load_and_process_documents(txt_path)
vector_db = create_vector_database(docs)

문서 로드 중: 100%|██████████| 104/104 [00:00<00:00, 2129.57it/s]


임베딩 모델 로드 중...
문서 분할 및 벡터 저장소 생성 중...


In [19]:
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

llm = ChatOllama(model='EEVE-korean-10.8B')

template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 한 문장으로 작성해 주세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

###예시
고소작업 시 추락 위험이 있는 부위에 안전장비 설치.
재발 방지 대책 마련과 안전교육 실시.
자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 후 장애물 사전 정리 작업 실시.

{context}

### 질문:
{question}

"""

prompt = ChatPromptTemplate.from_template(template)

def format_retrieved_docs(docs):
    """retriever가 반환한 문서를 문자열로 변환"""
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {
        'context': lambda inputs: format_retrieved_docs(retriever.get_relevant_documents(inputs['question'])),
        'question': itemgetter("question")
    }
    | prompt 
    | llm
    | StrOutputParser()
)

# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():
    # 50개당 한 번 진행 상황 출력
    # if (idx + 1) % 50 == 0 or idx == 0:
    #     print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")

    # RAG 체인 호출 및 결과 생성
    prevention_result = rag_chain.invoke({"question" : row['question']})
    print(prevention_result)
    if idx == 3:
        break

    # 결과 저장
    # result_text = prevention_result['result']
    test_results.append(prevention_result)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964
1. 재발 방지를 위한 안전대책:
   가. 지반 조사를 실시하고 필요한 경우 적절한 보강을 하여 펌프카의 지지력(아웃트리거)과 타설 위치를 확보하세요.
   나. 아웃트리거에 대한 정기적인 검사를 실시하여 안전하고 안정적인 상태를 유지하도록 합니다.
   다. 붐대호스의 최대 연장 길이를 제한하고 필요에 따라 호스를 조절하여 장비의 무게중심을 균형있게 유지하도록 합니다.
   라. 작업자들이 적절한 안전 교육을 받고 타설 중 발생할 수 있는 위험을 인지할 수 있도록 합니다.
2. 향후 조치 계획:
   가. 사고 조사 및 재발 방지를 위한 작업 절차 수립.
   나. 관련 인력에게 필요한 교육과 훈련 제공, 특히 지반 조건에 따른 타설 위치 선정과 붐대호스 조절에 대한 내용 포함.
   다. 정기적인 안전 점검 실시 및 안전 규정 준수를 위한 모니터링 강화.
   라. 사고 발생 시 적절한 대응을 위한 비상 절차 수립.
공정안전보건관리규칙 제2편 제4장 제1절(거푸집동바리 및 거푸집)에 의거하여 철골공사 현장에서의 무지보거푸집동바리(이하 '데크플레이트공법'이라 함) 설계, 조립 및 설치에 필요한 안전보건 작업 지침은 다음과 같습니다:

1. 목적
이지침은 산업안전보건기준에 관한 규칙(이하 '안전보건규칙'이라 함) 제2편 제4장 제1절(거푸집동바리 및 거푸집)의 규정에 따라 철골공사 현장에서 무지보거푸집동바리 데크플레이트공법에 필요한 안전보건 작업 지침을 정하기 위한 것입니다.

2. 적용 범위
이지침은 철골공사 현장에서의 거푸집동바리를 무지보거푸집동바리로 데크플레이트로 조립 및 설치하는 공사에 적용됩니다.

3. 용어의 정의
(1) 이 지침에서 사용되는 용어들은 다음과 같습니다:
- '데크플레이트(Deck plate)'란 아연도금강판, 선재 등 강재류를 요철가공하여 바닥 구조에 사용하는 파형으로 성형된 판을 말하며, 사다리꼴 모양 또는 사각형 모양의 단면을 가지고 있어 면외방향의 강성과 길이나압력을 가했을 

In [ ]:
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [4]:
import pandas as pd
modified_path  = "C:/wanted/Git_project/DACON-construction-accident-prevention/data/submission/JS/baseline_submission_20250323.csv"
modified_df  = pd.read_csv(modified_path, encoding="utf-8-sig")

In [5]:
missing_info = modified_df.isnull().sum()
print("결측값이 있는 컬럼:\n", missing_info[missing_info > 0])

결측값이 있는 컬럼:
 재발방지대책 및 향후조치계획    1
dtype: int64


In [6]:
for col in modified_df.columns:
    if modified_df[col].isnull().any():
        modified_df[col] = modified_df[col].fillna("" if modified_df[col].dtype == "object" else 0)

In [9]:
missing_info = modified_df.isnull().sum()
print("남은 결측값:\n", missing_info[missing_info > 0])

남은 결측값:
 Series([], dtype: int64)


In [19]:
modified_df.head()

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,"아웃트리거 지반 다짐 강화, 장비 설치 위치 변경, 무게 균형 확보, 작업 전 안전...",-1.424719,-0.660746,-0.046395,0.470930,-0.431656,1.105323,0.495366,-0.953521,...,0.684776,0.610682,0.158299,1.649820,0.645411,0.634861,0.103899,-0.326323,1.157921,0.091869
1,TEST_001,"절단작업 시 안전 교육 강화, 보안면 착용 의무화, 올바른 숫돌 사용법 교육, 작업...",0.020459,0.622030,-0.239208,0.611243,-0.564956,0.783140,0.831335,0.041809,...,0.905053,0.606571,0.247449,0.667848,0.663329,-0.039019,0.634422,-0.317569,0.680811,-0.389042
2,TEST_002,"작업 전 안전 점검 강화, 이동 경로 확보, 안전모 착용 의무화, 작업자 안전 교육...",-0.435971,0.625983,-0.775796,0.189448,-0.742293,1.009226,0.587601,-0.096395,...,0.503358,1.250370,-0.123251,1.098805,1.003068,-0.078078,-0.159424,-0.072634,0.861114,0.042451
3,TEST_003,"작업 발판 정리 정돈 및 안전 교육 강화, 미끄럼 방지 조치, 작업 전 안전 점검 ...",-0.035274,0.394262,-0.402448,0.075593,-0.562451,0.734280,0.598829,0.465471,...,0.571101,0.688688,0.017276,1.081550,0.425571,-0.008495,-0.164725,-0.150421,1.296279,-0.061823
4,TEST_004,"작업 전 안전교육 강화, 이동 경로 확보, 안전벨트 착용 의무화, 작업 감독 강화,...",-0.818977,0.371307,-0.255690,0.254254,-0.727086,0.682530,0.258270,-0.060448,...,0.443037,1.665349,0.415789,1.010559,0.841633,-0.954852,-0.812117,-0.005303,0.519075,0.171846


In [17]:
type(modified_df.iloc[0,3])

numpy.float64

In [7]:
modified_df.columns

Index(['ID', '재발방지대책 및 향후조치계획', 'vec_0', 'vec_1', 'vec_2', 'vec_3', 'vec_4',
       'vec_5', 'vec_6', 'vec_7',
       ...
       'vec_758', 'vec_759', 'vec_760', 'vec_761', 'vec_762', 'vec_763',
       'vec_764', 'vec_765', 'vec_766', 'vec_767'],
      dtype='object', length=770)

In [ ]:
output_path = "C:/wanted/Git_project/DACON-construction-accident-prevention/data/submission/JS"
modified_df.to_csv(f"{output_path}/Jacode_submission.csv", index=False, encoding="utf-8-sig")
print(f"저장 완료: {output_path}")

저장 완료: C:/wanted/Git_project/DACON-construction-accident-prevention/data/submission/JS
